In [82]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import warnings
import plotly.express as px

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from xgboost import XGBClassifier
import warnings

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go

train = pd.read_csv("../input/titanic/test.csv")
test = pd.read_csv("../input/titanic/train.csv")
df_data = pd.concat([train, test], sort=True).set_index('PassengerId')

In [83]:
df_data.head()

# <b>1 <span style='color:#F1C40F'>|</span> Missing Values</b>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>1.1 | Ticket</b></p>
</div>

Since data such as the **<span style='color:#F1C40F'>Ticket</span>** is useless, we proceed to drop their columns from the data frame.

In [84]:
cols_with_missing = [col for col in df_data.columns if df_data[col].isnull().any()] 
pd.isnull(df_data[cols_with_missing]).sum()

In [85]:
df_data = df_data.drop(['Ticket'],axis=1)

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>1.2 | Age</b></p>
</div>

To address the problem of missing values for the **<span style='color:#F1C40F'>Age</span>** field we will proceed as follows. Since **<span style='color:#F1C40F'>PClass</span>** is the variable that is **<span style='color:#F1C40F'>most correlated</span>** with both Age and Survived, we will group passengers according to the class they belong to. What we will do is replace the missing values with the **<span style='color:#F1C40F'>median</span>** of each group. In fact, what is more, within each of the existing classes we will make a **<span style='color:#F1C40F'>gender distinction</span>**. We do this because, as we will see below, the median of Age varies according to whether the passenger is male or female.

In [86]:
df_heatmap = pd.DataFrame(df_data.corr()['Age'].abs())
f,ax = plt.subplots(figsize=(10,1.5),facecolor='white')
sns.color_palette("rocket", as_cmap=True)          # Esta paleta es la que viene por defecto
sns.heatmap(df_heatmap.transpose(),annot = True,square=True, linewidths=1.5, cmap='rocket')

In [87]:
mediana = df_data.groupby(['Sex', 'Pclass']).median()['Age']
for i in range(0,mediana.shape[0]):
    if i<3: 
        print('Edad mediana para mujeres de la clase {}: {}'.format(i+1,mediana[i]))
    else:
        print('Edad mediana para hombres de la clase {}: {}'.format(i+1-3,mediana[i]))
df_data.Age = df_data.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))
print('Missing values for Age: {}'.format(df_data.Age.isnull().sum()))

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>1.3 | Embarked</b></p>
</div>

With respect to Embarked we will replace the missing data by the **<span style='color:#F1C40F'>mode</span>**, i.e. the most repeated value.

In [88]:
df_data.Embarked.value_counts()

In [89]:
moda = 'S'
df_data.Embarked = df_data.Embarked.replace(np.nan,moda)
pd.isnull(df_data).sum()

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>1.4 | Cabin</b></p>
</div>

**<span style='color:#F1C40F'>Cabin</span>** feature is little bit tricky and it needs further exploration. The large portion of the Cabin feature is missing and the feature itself **<span style='color:#F1C40F'>can't be ignored completely because some the cabins might have higher survival rates</span>**. It turns out to be the first letter of the Cabin values are the decks in which the cabins are located. Those decks were mainly separated for one passenger class, but some of them were used by multiple passenger classes.
![alt text](https://vignette.wikia.nocookie.net/titanic/images/f/f9/Titanic_side_plan.png/revision/latest?cb=20180322183733)
* On the Boat Deck there were **6** rooms labeled as **T, U, W, X, Y, Z** but only the **T** cabin is present in the dataset
* **A**, **B** and **C** decks were only for 1st class passengers
* **D** and **E** decks were for all classes
* **F** and **G** decks were for both 2nd and 3rd class passengers
* From going **A** to **G**, **<span style='color:#F1C40F'>distance to the staircase increases which might be a factor of survival</span>**

In [90]:
# Creating Deck column from the first letter of the Cabin column (M stands for Missing)
df_data['Deck'] = df_data['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'M')

df_data_decks = df_data.groupby(['Deck', 'Pclass']).count().drop(columns=['Survived', 'Sex', 'Age', 
                                                                        'Fare', 'Embarked', 'Cabin']).rename(columns={'Name': 'Count'}).transpose()

def get_pclass_dist(df):
    
    # Creating a dictionary for every passenger class count in every deck
    deck_counts = {'A': {}, 'B': {}, 'C': {}, 'D': {}, 'E': {}, 'F': {}, 'G': {}, 'M': {}, 'T': {}}
    decks = df.columns.levels[0]    
    
    for deck in decks:
        for pclass in range(1, 4):
            try:
                count = df[deck][pclass][0]
                deck_counts[deck][pclass] = count 
            except KeyError:
                deck_counts[deck][pclass] = 0
                
    df_decks = pd.DataFrame(deck_counts)    
    deck_percentages = {}

    # Creating a dictionary for every passenger class percentage in every deck
    for col in df_decks.columns:
        deck_percentages[col] = [(count / df_decks[col].sum()) * 100 for count in df_decks[col]]
        
    return deck_counts, deck_percentages

def display_pclass_dist(percentages):
    
    df_percentages = pd.DataFrame(percentages).transpose()
    deck_names = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'M', 'T')
    bar_count = np.arange(len(deck_names))  
    bar_width = 0.85
    
    pclass1 = df_percentages[0]
    pclass2 = df_percentages[1]
    pclass3 = df_percentages[2]
    
    plt.figure(figsize=(20, 10))
    plt.bar(bar_count, pclass1, color='#b5ffb9', edgecolor='white', width=bar_width, label='Passenger Class 1')
    plt.bar(bar_count, pclass2, bottom=pclass1, color='#f9bc86', edgecolor='white', width=bar_width, label='Passenger Class 2')
    plt.bar(bar_count, pclass3, bottom=pclass1 + pclass2, color='#a3acff', edgecolor='white', width=bar_width, label='Passenger Class 3')

    plt.xlabel('Deck', size=15, labelpad=20)
    plt.ylabel('Passenger Class Percentage', size=15, labelpad=20)
    plt.xticks(bar_count, deck_names)    
    plt.tick_params(axis='x', labelsize=15)
    plt.tick_params(axis='y', labelsize=15)
    
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), prop={'size': 15})
    plt.title('Passenger Class Distribution in Decks', size=18, y=1.05)   
    
    plt.show()    

all_deck_count, all_deck_per = get_pclass_dist(df_data_decks)
display_pclass_dist(all_deck_per)

📌 **Interpret:** for this graph we'll be grouping by **<span style='color:#F1C40F'>Deck</span>** and **<span style='color:#F1C40F'>PClass</span>** atribute. We are able to appreciate that A,B and C are fully occupied by passengers of 1st class. Moreover, as there is just one person in deck T, which class 1 we are going to group it with deck A. Deck of type D is mainly occupied with 1st class passengers, concretely a 85%. The rest are from 2nd class. To conclude, the remaining decks have all passengers from each of the classes.

In [91]:
# Passenger in the T deck is changed to A
idx = df_data[df_data['Deck'] == 'T'].index
df_data.loc[idx, 'Deck'] = 'A'

In [92]:
df_all_decks_survived = df_data.groupby(['Deck', 'Survived']).count().drop(columns=['Sex', 'Age', 'Fare', 
                                                                                   'Embarked', 'Pclass', 'Cabin']).rename(columns={'Name':'Count'}).transpose()

def get_survived_dist(df):
    
    # Creating a dictionary for every survival count in every deck
    surv_counts = {'A':{}, 'B':{}, 'C':{}, 'D':{}, 'E':{}, 'F':{}, 'G':{}, 'M':{}}
    decks = df.columns.levels[0]    

    for deck in decks:
        for survive in range(0, 2):
            surv_counts[deck][survive] = df[deck][survive][0]
            
    df_surv = pd.DataFrame(surv_counts)
    surv_percentages = {}

    for col in df_surv.columns:
        surv_percentages[col] = [(count / df_surv[col].sum()) * 100 for count in df_surv[col]]
        
    return surv_counts, surv_percentages

def display_surv_dist(percentages):
    
    df_survived_percentages = pd.DataFrame(percentages).transpose()
    deck_names = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'M')
    bar_count = np.arange(len(deck_names))  
    bar_width = 0.85    

    not_survived = df_survived_percentages[0]
    survived = df_survived_percentages[1]
    
    plt.figure(figsize=(20, 10))
    plt.bar(bar_count, not_survived, color='#b5ffb9', edgecolor='white', width=bar_width, label="Not Survived")
    plt.bar(bar_count, survived, bottom=not_survived, color='#f9bc86', edgecolor='white', width=bar_width, label="Survived")
 
    plt.xlabel('Deck', size=15, labelpad=20)
    plt.ylabel('Survival Percentage', size=15, labelpad=20)
    plt.xticks(bar_count, deck_names)    
    plt.tick_params(axis='x', labelsize=15)
    plt.tick_params(axis='y', labelsize=15)
    
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), prop={'size': 15})
    plt.title('Survival Percentage in Decks', size=18, y=1.05)
    
    plt.show()

all_surv_count, all_surv_per = get_survived_dist(df_all_decks_survived)
display_surv_dist(all_surv_per)

📌 **Interpret:** for this graph we'll be grouping by **<span style='color:#F1C40F'>Deck</span>** and **<span style='color:#F1C40F'>Survived</span>** atribute, in order to see the survival rate that each Deck has. We are able to appreciate that as expected survival rates are different for every type of Deck. **<span style='color:#F1C40F'>B</span>**, **<span style='color:#F1C40F'>D</span>** and **<span style='color:#F1C40F'>E</span>** are the ones with highest. On the other hand, **<span style='color:#F1C40F'>A</span>** and **<span style='color:#F1C40F'>M</span>** are the ones with lowest. 

Due to what we have just seen before, we are going to label decks in the following way: 
* A, B and C decks, as they all have 1st class passengers, are going to be labeled as ABC
* **D** and **E** decks are labeled as **DE** because both of them have similar passenger class distribution and same survival rate
* Following the previous criterion we labeled FG
* M remains equal because it's quite different from the others and it's the one with lowest survival rate.

In [93]:
df_data['Deck'] = df_data['Deck'].replace(['A', 'B', 'C'], 'ABC')
df_data['Deck'] = df_data['Deck'].replace(['D', 'E'], 'DE')
df_data['Deck'] = df_data['Deck'].replace(['F', 'G'], 'FG')

df_data['Deck'].value_counts()

In [94]:
df_data = df_data.drop('Cabin',axis=1)
df_data.isnull().sum()

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>1.5 | Fare</b></p>
</div>

We have one missing value for **<span style='color:#F1C40F'>Fare</span>**, belonging to the test dataset. We are going to fill it with the most repeated value of this field.

In [95]:
tst = df_data[df_data.Survived.isnull() == True]
mediana = tst.Fare.describe()[6]
df_data.Fare = df_data.Fare.fillna(mediana)

# <b>2 <span style='color:#F1C40F'>|</span> Feature Engineering</b>

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>2.1 | Family</b></p>
</div>

We will start by creating fields related to the family unit. The first of these will come from the **<span style='color:#F1C40F'>SibSp and Parch</span>** fields, which we can remove later. This will reflect the **<span style='color:#F1C40F'>size of passengers' family</span>**. We will also enter a field to indicate whether the passenger is travelling **<span style='color:#F1C40F'>alone</span>** or not.

In [96]:
df_data['FamilySize'] = df_data.Parch + df_data.SibSp + 1
df_data['IsAlone'] = 0
df_data.loc[df_data['FamilySize'] == 1, 'IsAlone'] = 1
df_data = df_data.drop(['Parch','SibSp'],axis = 1)

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>2.2 | Passenger's Name</b></p>
</div>

Next we are going to add a column that will be, in part, related to the **<span style='color:#F1C40F'>Name</span>** field:

In [97]:
df_data.head()

In [98]:
df_data['Title'] = df_data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(df_data['Title'], df_data['Sex']).transpose()

We can replace many titles with a more common name or classify them as Rare.

In [99]:
df_data['Title'] = df_data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

df_data['Title'] = df_data['Title'].replace('Mlle', 'Miss')
df_data['Title'] = df_data['Title'].replace('Ms', 'Miss')
df_data['Title'] = df_data['Title'].replace('Mme', 'Mrs')

df_data[['Title', 'Survived']].groupby(['Title'], as_index=False).mean().transpose()

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>2.3 | Fare</b></p>
</div>

In order to binning continuous features we are going to use **<span style='color:#F1C40F'>13 quantile base bins</span>**. Even though the bins are too much, they provide a decent amount of information gain, as it would be seen in next section. We'll create `df_data_no_quart` in order to have a DataFrame with the discrete values of Fare, to make it easier to plotting it later.

In [100]:
df_data_no_quart = df_data.copy()
names = ['1', '2', '3', '4', '5', '6', '7','8','9','10','11','12','13']
df_data['Fare'] = pd.qcut(df_data['Fare'], 13, labels = names)
df_data.Fare = pd.to_numeric(df_data.Fare, errors = 'coerce')

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>2.4 | Age</b></p>
</div>

Let's keep binning continuous features. For **<span style='color:#F1C40F'>Age</span>** we are going to use **<span style='color:#F1C40F'>10 quantile base bins</span>**. Even though the bins are too much, they provide a decent amount of information gain, as it would be seen in next section.

In [101]:
names = ['1','2','3','4','5','6','7','8','9','10']
df_data['Age'] = pd.qcut(df_data['Age'], 10, labels = names)

# <b>3 <span style='color:#F1C40F'>|</span> Data Visualization</b>

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>3.1 | Heat Map</b></p>
</div>

📌 **Interpret:** On the one hand, the variables with the highest correlations are: **<span style='color:#F1C40F'>PClass</span>** and **<span style='color:#F1C40F'>Fare</span>**. On the other hand, we can appreciate that both, Age and FamilySize are the ones with less correlation coefficient with respect to Survived.

In [102]:
df_heatmap = pd.DataFrame(df_data.corr()['Survived'].abs())
f,ax = plt.subplots(figsize=(16,1.5),facecolor='white')
sns.color_palette("rocket", as_cmap=True)          # Esta paleta es la que viene por defecto
sns.heatmap(df_heatmap.transpose(),annot = True,square=True, linewidths=1.5, cmap='rocket')

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>3.2 | Group of Age Visualization</b></p>
</div>

📌 **Interpret:** We have created several **<span style='color:#F1C40F'>age groups</span>**, in order to make it easier to make the graphs. On the left size, we have the average fare per each group of age. As observed, **<span style='color:#F1C40F'>older group (48-80)</span>** is the group which has payed more. At the bottom part of the bar graph we can find groups to which teenagers and young adults belong. On the right, I have made a pie plot in order to find which percentage of people from each group survive, and people from which group is more likely to survive. We can see that **<span style='color:#F1C40F'>kids (0.169-16)</span>** are the ones with more survival rate (14.6%). Moreover, we also observe that half of the kids survived. Concretely, a 58.8% of kids survived. On the other hand, older people and youngsters between 21-22 years old bear the brunt as they are the least likely to survive. Indeed, from the whole group of old people just 34% survived aproximately.

In [103]:
#names = ['0-8', '9-15', '16-18', '19-25', '26-40', '41-60', '61-100']
oneHot_train_graph = df_data_no_quart.copy()
names = ['0.169-16','16-21','21-22','22-25','25-26','26-29.5','29.5-34','34-40','40-48','48-80']
oneHot_train_graph['Age'] = pd.qcut(oneHot_train_graph['Age'], 10, labels = names)
df_bar = oneHot_train_graph.groupby('Age').agg({'Fare':'mean'}).reset_index().sort_values(by='Fare',ascending=False).set_index('Age')
df_pie = oneHot_train_graph.groupby('Age').agg({"Survived" : "mean"}).reset_index().sort_values(by='Survived', ascending=False).set_index('Age')

fig = make_subplots(rows=1, cols=2, 
                    specs=[[{"type": "bar"}, {"type": "pie"}]],                          
                    column_widths=[0.7, 0.3], vertical_spacing=0, horizontal_spacing=0.02,
                    subplot_titles=("Average Fare per Group of Age", "Survival Percentage per Group of Age"))

fig.add_trace(go.Bar(x=df_bar['Fare'], y=df_bar.index, marker=dict(color=['#334550','#334550','#394184','#394184','#6D83AA','#6D83AA','#C8D0DF','#C8D0DF','#C8D0DF','#C8D0DF']),
                     name='Fare', orientation='h'), 
                     row=1, col=1)
fig.add_trace(go.Pie(values=df_pie['Survived'], labels=df_pie.index, name='Age',
                     marker=dict(colors=['#334550','#334668','#394184','#496595','#6D83AA','#91A2BF','#C8D0DF']), hole=0.7,
                     hoverinfo='label+percent+value', textinfo='label'), 
                    row=1, col=2)
# styling
fig.update_yaxes(showgrid=False, ticksuffix=' ', categoryorder='total ascending', row=1, col=1)
fig.update_xaxes(visible=False, row=1, col=1)
fig.update_layout(height=500, bargap=0.2,
                  margin=dict(b=0,r=20,l=20), xaxis=dict(tickmode='linear'),
                  title_text="Group of Age Analysis",
                  template="plotly_white",
                  title_font=dict(size=29, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#8a8d93'), 
                  hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"),
                  showlegend=False)
fig.show()

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>3.3 | PClass Visualization</b></p>
</div>

📌 **Interpret:** for this graph we'll be grouping by **<span style='color:#F1C40F'>PClass</span>** atribute. I have plotted 3 different graphs relating PClass with **<span style='color:#F1C40F'>Title</span>**, **<span style='color:#F1C40F'>Loneliness</span>** and **<span style='color:#F1C40F'>Family Size</span>**. We are able to appreciate that **<span style='color:#F1C40F'>lonely people</span>** have chosen class 3 more than any other. Indeed, both Class 3 and Class 2 have been the preference for more lonely people than accompanied. Class 1 as seen, is more likely to be chosen by people accompanied by one familiar. Families with 3 or more people on board are uniformly distributed between different classes.

In [104]:
fig, axes = plt.subplots(nrows=1, ncols=3,figsize=(20, 8))

sns.set_style('whitegrid')
ax = sns.countplot(x = "Pclass", hue='Title', data = df_data,ax = axes[0], palette=['#334550','#394184','#6D83AA','#91A2BF','#C8D0DF']);
ax.set_title('PClass per Title')

sns.set_style('whitegrid')
ax = sns.countplot(x = "Pclass", hue='IsAlone', data = df_data,ax = axes[1], palette=['#334550','#C8D0DF']);
_ = ax.set_title('PClass per Loneliness')

sns.set_style('whitegrid')
ax = sns.countplot(x = "Pclass", hue='FamilySize', data = df_data,ax = axes[2], palette=['#334550','#394184','#6D83AA','#91A2BF','#C8D0DF']);
__ = ax.set_title('PClass per Family Size')

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>3.4 | Fare Visualization</b></p>
</div>

📌 **Interpret:** The groups at the left side of the graph has the lowest survival rate and the groups at the right side of the graph has the highest survival rate. This high survival rate was not visible in the distribution graph. There is also an unusual group (15.742, 23.25] in the middle with high survival rate that is captured in this process.

In [105]:
fig, axs = plt.subplots(figsize=(22, 9))
sns.countplot(x='Fare', hue='Survived', data=df_data, palette=['#334550','#C8D0DF'])

plt.xlabel('Fare', size=15, labelpad=20)
plt.ylabel('Passenger Count', size=15, labelpad=20)
plt.tick_params(axis='x', labelsize=10)
plt.tick_params(axis='y', labelsize=15)

plt.legend(['Not Survived', 'Survived'], loc='upper right', prop={'size': 15})
plt.title('Count of Survival in {} Feature'.format('Fare'), size=15, y=1.05)

plt.show()

# <b>4 <span style='color:#F1C40F'>|</span> Categorical Variables</b>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>4.1 | One Hot Encoding</b></p>
</div>

We will now proceed with the oneHot Imputation. With this, we will **<span style='color:#F1C40F'>change the values of the categorical variables to numerical</span>**. This technique comes in handy since both categorical variables have a **<span style='color:#F1C40F'>small</span>** number of unique values.

In [106]:
df_data.dtypes

In [107]:
print(df_data.Embarked.unique())
print(df_data.Sex.unique())

In [108]:
object_cols = [cname for cname in df_data.columns if df_data[cname].dtype == "object"]
object_cols

In [109]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_data = pd.DataFrame(OH_encoder.fit_transform(df_data[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_data.index = df_data.index

# Remove categorical columns (will replace with one-hot encoding)
num_x_data = df_data.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_data= pd.concat([num_x_data, OH_cols_data], axis=1)

In [110]:
OH_data

As we can see, columns **<span style='color:#F1C40F'>0</span>** and **<span style='color:#F1C40F'>1</span>** correspond to the **<span style='color:#F1C40F'>Sex</span>** attribute, while the rest with **<span style='color:#F1C40F'>Embarked</span>**. At this point, we have our dataset ready to be modeled.

# <b>5 <span style='color:#F1C40F'>|</span> Modeling</b>
<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>5.1 | Cross Validation</b></p>
</div>

For the modeling part we will compare 10 known algorithms, and proceed to evaluate their average accuracy by a **<span style='color:#F1C40F'>stratified kfold cross validation</span>** procedure:
* SVC
* Decision Tree
* AdaBoost
* Random Forest
* Extra Trees
* Gradient Boosting
* Multiple layer perceprton (neural network)
* KNN
* Logistic regression
* Linear Discriminant Analysis
* XGBoost Classifier

To begin with, we are going to create a cross validate model with Kfold stratified. Then we'll test each of the algorithms that I have mentioned before.

In [111]:
x_train = OH_data[OH_data.Survived.isnull() == False].drop('Survived',axis=1)
y_train = OH_data[OH_data.Survived.isnull() == False].Survived

In [112]:
kfold = StratifiedKFold(n_splits=10)
random_state = 2
classifiers = []
classifiers.append(SVC(random_state=random_state))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LinearDiscriminantAnalysis())
classifiers.append(XGBClassifier(random_state = random_state))

cv_results = []
for classifier in classifiers :
    cv_results.append(cross_val_score(classifier, x_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=4))
    
cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())
    
cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVC","DecisionTree","AdaBoost",
"RandomForest","ExtraTrees","GradientBoosting","MultipleLayerPerceptron","KNeighboors","LinearDiscriminantAnalysis",'XGBClassifier']})
cv_res = cv_res.sort_values(by='CrossValMeans',ascending = False)

In [113]:
fig = make_subplots(rows=1, cols=1, 
                    specs=[[{"type": "bar"}]])

fig.add_trace(go.Bar(x=cv_res['CrossValMeans'], y=cv_res.Algorithm, marker=dict(color=['#334550','#334550','#334668','#334668','#496595','#496595','#6D83AA','#6D83AA','#91A2BF','#C8D0DF']),
                     name='Fare', orientation='h'), 
                     row=1, col=1)
# styling
fig.update_yaxes(showgrid=True, ticksuffix=' ', categoryorder='total ascending', row=1, col=1)
fig.update_xaxes(visible=True, row=1, col=1)
fig.update_layout(height=500, bargap=0.1,
                  margin=dict(b=0,r=20,l=20), xaxis=dict(tickmode='linear'),
                  title_text="Cross Validation Scores",
                  template="plotly_white",
                  title_font=dict(size=29, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#8a8d93'), 
                  hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"),
                  showlegend=False)
fig.show()

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>5.2 | Hyperparameter tuning</b></p>
</div>

For the ensemble modeling we are going to use: 
* ExtraTreesClassifier
* SVC
* AdaBoost 
* RandomForest
* GradientBoosting
In order to make execution quicker, we set **<span style='color:#F1C40F'>n_jobs to -1</span>**. This means that we are going to use every CPU we have in the computer.

In [114]:
# Adaboost
DTC = DecisionTreeClassifier()
adaDTC = AdaBoostClassifier(DTC, random_state=7)
ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}
gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="accuracy", n_jobs= -1, verbose = 1)
gsadaDTC.fit(x_train,y_train)
ada_best = gsadaDTC.best_estimator_

# RFC Parameters tunning 
RFC = RandomForestClassifier()
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}
gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= -1, verbose = 1)
gsRFC.fit(x_train,y_train)
RFC_best = gsRFC.best_estimator_

# Gradient boosting tunning
GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1]}
gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs= -1, verbose = 1)
gsGBC.fit(x_train,y_train)
GBC_best = gsGBC.best_estimator_

# SVC classifier
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}
gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= -1, verbose = 1)
gsSVMC.fit(x_train,y_train)
SVMC_best = gsSVMC.best_estimator_

#ExtraTrees 
ExtC = ExtraTreesClassifier()
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}
gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs= -1, verbose = 1)
gsExtC.fit(x_train,y_train)
ExtC_best = gsExtC.best_estimator_

<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>5.3 | Ensemble Modeling and Prediction</b></p>
</div>

For the final part of this project I have chosen **<span style='color:#F1C40F'>VotingClassifier</span>**. We'll fit the model and then proceed to make the predictions. At the final part of this section you'll find some graphs related to survival predictions made.

In [115]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best),
('svc', SVMC_best), ('adac',ada_best),('gbc',GBC_best)], voting='soft', n_jobs=4)

votingC = votingC.fit(x_train, y_train)

In [116]:
x_test = OH_data[OH_data.Survived.isnull() == True].drop('Survived',axis=1)
mediana = x_test.Fare.describe()[6]
x_test.Fare = x_test.Fare.fillna(mediana)
predictions_survived = votingC.predict(x_test)

In [117]:
predictions = pd.DataFrame({'Survived' : predictions_survived},index = x_test.index)
predictions['Survived'] = predictions.Survived.astype(int)
predictions.to_csv('submission.csv')

Finally, we will graph the results obtained in order to illustrate the understanding of these results.

In [118]:
x_test_predicted = df_data[df_data.Survived.isnull() == True].copy()
x_test_predicted['Survived'] = predictions.Survived
df_barras_sexo = pd.DataFrame({'Survived':x_test_predicted.Survived, 'Sex':x_test_predicted.Sex, 'Fare':x_test_predicted.Fare,'PClass':x_test_predicted.Pclass,'Age':x_test_predicted.Age,'IsAlone':x_test_predicted.IsAlone,'Title':x_test_predicted.Title})
df_barras_sexo['Survived'].replace([0,1], ['No sobrevive','Sobrevive'], inplace=True)

fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(20, 8))

sns.set_style('whitegrid')
ax = sns.countplot(x = "Survived", hue='Sex', data = df_barras_sexo,ax = axes[0],palette=['#334550','#C8D0DF'])
ax.set_title('Cantidad de supervivientes/no supervivientes')

sns.set_style('whitegrid')
ax = sns.barplot(x = "Survived", y='Fare', hue = 'Sex', data = df_barras_sexo,ax = axes[1],palette=['#394184','#C8D0DF']);
_ = ax.set_title('Relación Supervivencia/Tarifa por Sexos')

In [119]:
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(22, 11))

sns.set_style('whitegrid')
ax = sns.countplot(x = "PClass", hue='Survived', data = df_barras_sexo,ax = axes[0],palette=['#334550','#C8D0DF']);
ax.set_title('Cantidad de supervivientes/no supervivientes según PClass')

sns.set_style('whitegrid')
ax = sns.swarmplot(x = "Survived", y='Age', hue = 'Sex', data = df_barras_sexo,ax = axes[1],palette=['#334550','#C8D0DF']);
_ = ax.set_title('Relación Supervivencia/Edad por Sexos')

In [120]:
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(20, 8))

sns.set_style('whitegrid')
ax = sns.countplot(x = "Survived", hue='Title', data = df_barras_sexo,ax = axes[0], palette=['#334550','#394184','#6D83AA','#91A2BF','#C8D0DF']);
ax.set_title('PClass per Title')

sns.set_style('whitegrid')
ax = sns.countplot(x = "Survived", hue='IsAlone', data = df_barras_sexo,ax = axes[1], palette=['#334550','#C8D0DF']);
_ = ax.set_title('PClass per Loneliness')